# DAT_SF_13 - Lab 12

### Recommender Systems


1. Simple similarity based recommendation system
2. Recsys


## Similarity based Recommendation System: Beers


Let's build a recommendation system to recommend types of beers based on user reviews

Usual imports (numpy, pandas)

In [1]:
import pandas as pd
import numpy as np

First of all let's get the data

In [2]:
! curl -O https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.3M  100 27.3M    0     0  2587k      0  0:00:10  0:00:10 --:--:-- 3045k


Import data in a pandas dataframe called "allbeers". Use the compression keyword

In [6]:
allbeers = pd.read_csv('beer_reviews.tar.gz',compression = 'gzip')

Let's look at the data

In [7]:
allbeers.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [58]:
allbeers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586629 entries, 0 to 1586628
Data columns (total 13 columns):
beer_reviews/         1586629 non-null object
brewery_name          1586611 non-null object
review_time           1586614 non-null float64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null float64
dtypes: float64(8), object(5)
memory usage: 169.5+ MB


Let's restrict this to the top 250 beers. Use the value_counts() method select the top 250 beers.
Assign the selected beers to a dataset called df

In [19]:
n = 250
top_n = allbeers.beer_name.value_counts().index[:n]
df = allbeers[allbeers.beer_name.isin(top_n)]

In [59]:
top_n

Index([u'90 Minute IPA', u'India Pale Ale',
       u'Old Rasputin Russian Imperial Stout',
       u'Sierra Nevada Celebration Ale', u'Two Hearted Ale',
       u'Stone Ruination IPA', u'Arrogant Bastard Ale',
       u'Sierra Nevada Pale Ale', u'Stone IPA (India Pale Ale)',
       u'Pliny The Elder', 
       ...
       u'Murphy's Irish Stout', u'Troegenator Double Bock',
       u'Aecht Schlenkerla Rauchbier Märzen', u'Smuttynose Robust Porter',
       u'Piraat Ale', u'Raging Bitch Belgian-Style IPA', u'Stone Pale Ale',
       u'Hopsickle Imperial India Pale Ale',
       u'New Holland Dragon's Milk Oak Barrel Ale', u'Vanilla Porter'],
      dtype='object', length=250)

In [60]:
df.head()


,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
798,1075,Caldera Brewing Company,1212201268,4.5,4.5,4,grumpy,American Double / Imperial Stout,4.0,4.5,Imperial Stout,NaN,42964
1559,11715,Destiny Brewing Company,1137124057,4.0,3.5,4,blitheringidiot,American Pale Ale (APA),3.5,3.5,Pale Ale,4.5,26420
1560,11715,Destiny Brewing Company,1129504403,4.0,2.5,4,NeroFiddled,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,26420
1563,11715,Destiny Brewing Company,1137125989,3.5,3.0,4,blitheringidiot,American IPA,4.0,4.0,IPA,NaN,26132
1564,11715,Destiny Brewing Company,1130936611,3.0,3.0,3,Gavage,American IPA,4.0,3.5,IPA,NaN,26132


How big is this dataset?

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355275 entries, 798 to 1586564
Data columns (total 13 columns):
beer_reviews/         355275 non-null object
brewery_name          355275 non-null object
review_time           355275 non-null float64
review_overall        355275 non-null float64
review_aroma          355275 non-null float64
review_appearance     355275 non-null float64
review_profilename    355175 non-null object
beer_style            355275 non-null object
review_palate         355275 non-null float64
review_taste          355275 non-null float64
beer_name             355275 non-null object
beer_abv              353477 non-null float64
beer_beerid           355275 non-null float64
dtypes: float64(8), object(5)
memory usage: 37.9+ MB


### Pivot Table

Aggregate the data in a pivot table using the pivot_table method. Display the mean review_overall for each beer_name aggregating the review_overall values by review_profilename. Use the mean as aggregator.

In [62]:
df_wide = pd.pivot_table(df,values = ['review_overall'], 
                         columns = ['beer_name', 'review_profilename'], aggfunc =np.mean).unstack()

In [28]:
df_wide.shape

(250, 22140)

Display the head of the pivot table, but only for 5 users (columns are users)

In [29]:
df_wide.ix[0:5, 0:5]

review_profilename                         0110x011  02maxima  03SVTCobra  \
               beer_name                                                    
review_overall #9                               NaN       NaN         NaN   
               120 Minute IPA                   NaN       NaN         NaN   
               1554 Enlightened Black Ale       NaN       NaN         NaN   
               60 Minute IPA                    NaN       NaN         NaN   
               90 Minute IPA                      5       NaN         NaN   

review_profilename                         05Harley  0Naught0  
               beer_name                                       
review_overall #9                               NaN       NaN  
               120 Minute IPA                     4       NaN  
               1554 Enlightened Black Ale       NaN       NaN  
               60 Minute IPA                    NaN       NaN  
               90 Minute IPA                      4       NaN

### Discussion: what do you notice in this table?

#### Data munging
Set Nans to zero

In [63]:
df_wide = df_wide.fillna(0)

Check that columns are users

In [64]:
df_wide.columns[:10]

Index([u'0110x011', u'02maxima', u'03SVTCobra', u'05Harley', u'0Naught0',
       u'0beerguy0', u'0runkp0s', u'0tt0', u'1000Bottles', u'1001111.0'],
      dtype='object', name=u'review_profilename')

Check that rows are beers

In [34]:
df_wide.index[:10]

MultiIndex(levels=[[u'review_overall'], [u'#9', u'120 Minute IPA', u'1554 Enlightened Black Ale', u'60 Minute IPA', u'90 Minute IPA', u'Aecht Schlenkerla Rauchbier Märzen', u'AleSmith IPA', u'AleSmith Speedway Stout', u'Allagash White', u'Alpha King Pale Ale', u'Anchor Liberty Ale', u'Anchor Old Foghorn', u'Anchor Porter', u'Anchor Steam Beer', u'ApriHop', u'Arrogant Bastard Ale', u'Ayinger Celebrator Doppelbock', u'Bass Pale Ale', u'Bell's Expedition Stout', u'Bell's Hopslam Ale', u'Bell's Java Stout', u'Bell's Kalamazoo Stout', u'Bell's Oberon Ale', u'Bell's Special Double Cream Stout', u'Big Bear Black Stout', u'Black Butte Porter', u'Blanche De Chambly', u'Blind Pig IPA', u'Blue Moon Belgian White', u'Boddingtons Pub Ale', u'Bourbon County Brand Stout', u'Brooklyn Black Chocolate Stout', u'Brooklyn Lager', u'Brooklyn Monster Ale', u'Brown Shugga'', u'Brutal Bitter Ale', u'Bud Light', u'Budweiser', u'Burton Baton', u'Cappuccino Stout', u'Chicory Stout', u'Chimay Grande Réserve (Blue

### Calculate distance between beers

We're going to use cosine_similarity from scikit-learn to compute the distance between all beers

Imports

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

Apply cosine similarity to df_wide to calculate pairwise distances

In [37]:
dists = cosine_similarity(df_wide)

### Discussion: what type of object is dists?

Convert dists to a Pandas DataFrame, use the index as column index as well (distances are a square matrix).

In [38]:
dists = pd.DataFrame(dists, columns = df_wide.index)
dists.index = dists.columns

In [40]:
dists.ix[:10,:10]

review_overall  \
beer_name                                                     #9   
               beer_name                                           
review_overall #9                                       1.000000   
               120 Minute IPA                           0.275405   
               1554 Enlightened Black Ale               0.274103   
               60 Minute IPA                            0.388364   
               90 Minute IPA                            0.365175   
               Aecht Schlenkerla Rauchbier Märzen       0.253841   
               AleSmith IPA                             0.228479   
               AleSmith Speedway Stout                  0.227612   
               Allagash White                           0.340681   
               Alpha King Pale Ale                      0.293315   

                                                                  \
beer_name                                         120 Minute IPA   
               beer_name                                           
review_overall #9                                       0.275405   
               120 Minute IPA                           1.000000   
               1554 Enlightened Black Ale               0.251519   
               60 Minute IPA                            0.378258   
               90 Minute IPA                            0.410366   
               Aecht Schlenkerla Rauchbier Märzen       0.262425   
               AleSmith IPA                             0.315971   
               AleSmith Speedway Stout                  0.337541   
               Allagash White                           0.282273   
               Alpha King Pale Ale                      0.336796   

                                                                              \
beer_name                                         1554 Enlightened Black Ale   
               beer_name                                                       
review_overall #9                                                   0.274103   
               120 Minute IPA                                       0.251519   
               1554 Enlightened Black Ale                           1.000000   
               60 Minute IPA                                        0.319887   
               90 Minute IPA                                        0.314028   
               Aecht Schlenkerla Rauchbier Märzen                   0.252486   
               AleSmith IPA                                         0.266866   
               AleSmith Speedway Stout                              0.261761   
               Allagash White                                       0.260275   
               Alpha King Pale Ale                                  0.307296   

                                                                               \
beer_name                                         60 Minute IPA 90 Minute IPA   
               beer_name                                                        
review_overall #9                                      0.388364      0.365175   
               120 Minute IPA                          0.378258      0.410366   
               1554 Enlightened Black Ale              0.319887      0.314028   
               60 Minute IPA                           1.000000      0.533042   
               90 Minute IPA                           0.533042      1.000000   
               Aecht Schlenkerla Rauchbier Märzen      0.316928      0.312861   
               AleSmith IPA                            0.312343      0.344218   
               AleSmith Speedway Stout                 0.307627      0.358754   
               Allagash White                          0.360975      0.356804   
               Alpha King Pale Ale                     0.385249      0.418582   

                                                                                      \
beer_name                                         Aecht Schlenkerla Rauchbier Märzen   
         

Select some beers and look their distances to other beers

In [66]:
some_beers = ['AleSmith IPA', 'Allagash White', '120 Minute IPA']

TypeError: cannot perform reduce with flexible type

Sum the distances of my favourite beers by row, to have one distance from each beer in the sample

In [47]:
beer_summed = dists.apply(lambda row: np.sum(row), axis =1)

#### Performance

Optional: which one is faster? use ```%timeit``` to check

In [52]:
%timeit dists.apply(lambda row: np.sum(row), axis =1)

10 loops, best of 3: 18.2 ms per loop


In [51]:
%timeit np.sum(dists, axis =1)

1000 loops, best of 3: 337 µs per loop


#### Ranking

Sort summed beers from best to worse

In [48]:
beer_summed = beer_summed.order(ascending = False)

In [49]:
beer_summed.head()

                beer_name                                 
review_overall  Storm King Stout                              95.661660
                Sierra Nevada Celebration Ale                 94.964305
                Stone IPA (India Pale Ale)                    94.700958
                Sierra Nevada Bigfoot Barleywine Style Ale    94.676854
                Hop Rod Rye                                   94.618248
dtype: float64

Filter out the beers used as input and transform to list

In [57]:
ranked_beers = beer_summed.tolist()
ranked_beers[:5]

[95.661659884219276,
 94.964304982769647,
 94.700957998207514,
 94.676853918693652,
 94.618248079200811]

Define a function that does what we just did for an arbitrary input list of beers. it should also receive the maximum number of beers requested n as optional parameter.

Test your function. Find the 10 beers most similar to "120 Minute IPA"

Cool, let's try again with the 10 beers most similar to ["Coors Light", "Bud Light", "Amstel Light"]

Optional: register an account on yhat and deploy your model following the instructions [here](https://docs.yhathq.com/python/examples/beer-recommender) and [here](http://nbviewer.ipython.org/gist/glamp/20a18d52c539b87de2af)

## Recsys

A python library for implementing a recommender system

In [ ]:
"""
##install python-recsys

### first install dependencies

pip install csc-pysparse networkx divisi2

### then install recsys
git clone https://github.com/python-recsys/python-recsys.git
cd python-recsys/

python setup.py install
"""

Load recsys.algotihm, set VERBOSE = True import SVD class

In [ ]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD

Let's look at the files

In [ ]:
! ls movielens

Import 'movies.dat' to a 'movies' pandas dataframe. Make sure you name the columns, use the correct separator and define the index.

In [ ]:
movies.head()

Import 'ratings.dat' to a 'ratings' pandas dataframe. Make sure you name the columns, use the correct separator.

In [ ]:
ratings.head()

Initialize an SVD instance

In [ ]:
svd = SVD()

Populate it with the data from the ratings dataset, using the built in load_data method

Compute SVD

$M=U \Sigma V^T$:

In [ ]:
k = 100


you can also save the output SVD model (in a zip file)

In [ ]:
# svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True, savefile='/tmp/movielens')

Reload a saved model:

In [ ]:
# svd2 = SVD(filename='/tmp/movielens')

Find the ITEMID number for "Toy Story (1995)"

Find the ITEMID number for "Bug's Life, A (1998)"

Compute similarity between the two movies

Get movies similar to Toy Story

Predict rating for a given user and movie, $\hat{r}_{ui}$

Recommend non rated movies to a user

Which users should see Toy Story? (e.g. which users -that have not rated Toy Story- would give it a high rating?)

Find out more here: [https://github.com/ocelma/python-recsys](https://github.com/ocelma/python-recsys)